Code for expanding the lexicon

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv
from xml.dom.minidom import Document


In [1]:
import csv

def convert_text_to_csv(input_file, output_file):
    # Open the input text file for reading
    with open(input_file, 'r', encoding='utf-8') as f:
        # Skip lines until the first line starting with underscore
        for line in f:
            if line.startswith('_'):
                break
        
        # Initialize a list to store the data
        data = []
        
        # Read the remaining lines
        for line in f:
            # Split the line by tabs and remove leading/trailing whitespace
            columns = line.strip().split('\t')
            
            # Append the columns to the data list
            data.append(columns)

    # Open the output CSV file for writing
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        # Create a CSV writer object
        writer = csv.writer(f)
        
        # Write the data to the CSV file
        writer.writerows(data)

    print("Conversion completed. CSV file saved as", output_file)

# Example usage:
input_file_path = 'saldo20v03.txt'
output_file_path = 'saldo20v03CSV.csv'
convert_text_to_csv(input_file_path, output_file_path)

Conversion completed. CSV file saved as saldo20v03CSV.csv


In [8]:
import csv

# Path to the SenSALDO text file (as uploaded)
sensaldo_path = 'sensaldo-base-v02.txt'

# Path to the output CSV file
sensaldo_csv_path = 'sensaldo.csv'

# Open the SenSALDO file and create a CSV file for output
with open(sensaldo_path, 'r', encoding='utf-8') as infile, \
     open(sensaldo_csv_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    
    # Write the headers for the CSV file
    headers = ['SALDO sense ID', 'polarity label']
    writer.writerow(headers)
    
    # Read the SenSALDO file line by line
    for line in infile:
        # Skip empty lines and lines starting with '#' (comments)
        if not line.strip() or line.startswith('#'):
            continue
        
        # Split the line into components based on tabs
        sense_id, polarity = line.strip().split('\t')
        
        # Write the components to the CSV file
        writer.writerow([sense_id, polarity])

print(f"Conversion complete. The CSV file is saved at {sensaldo_csv_path}.")

Conversion complete. The CSV file is saved at sensaldo.csv.


Expansion

In [29]:
saldoDf = pd.read_csv('saldo20v03CSV.csv', names=['betydelse', 'lemgram', 'ordklass', 'prim', 'sek', 'barnPrim', 'barnSek' ], encoding='utf-8')
sensaldoDf = pd.read_csv('sensaldo.csv', encoding='utf-8')

In [30]:
saldoDfC = saldoDf.copy().drop_duplicates()
sensaldoDfC = sensaldoDf.copy().drop_duplicates()

In [31]:
saldoDf.head()

,betydelse,lemgram,ordklass,prim,sek,barnPrim,barnSek
0,_1984..1,roman..1,dystopi..1,nittonhundraåttiofyra..pm.1,nittonhundraåttiofyra,pm,pm_uwb_hemsöborna
1,_1984..1,roman..1,dystopi..1,Nittonhundraåttiofyra..pm.1,Nittonhundraåttiofyra,pm,pm_uwb_hemsöborna
2,a..1,bokstav..1,PRIM..1,a..nn.1,a,nn,nn_vn_alfa_abc
3,A..1,initial..1,PRIM..1,A..nn.1,A,nn,nn_vn_alfa_abc
4,à..1,per..1,PRIM..1,á..pp.1,á,pp,pp_i_i


In [33]:
saldoDfC.loc[saldoDfC['betydelse']=='abborre..1']



,betydelse,lemgram,ordklass,prim,sek,barnPrim,barnSek
27,abborre..1,fisk..1,PRIM..1,abborre..nn.1,abborre,nn,nn_2u_vinge
28,abborre..1,fisk..1,PRIM..1,aborre..nn.1,aborre,nn,nn_2u_vinge


In [11]:
saldoDfC

,_1984..1,roman..1,dystopi..1,nittonhundraåttiofyra..pm.1,nittonhundraåttiofyra,pm,pm_uwb_hemsöborna
0,_1984..1,roman..1,dystopi..1,Nittonhundraåttiofyra..pm.1,Nittonhundraåttiofyra,pm,pm_uwb_hemsöborna
1,a..1,bokstav..1,PRIM..1,a..nn.1,a,nn,nn_vn_alfa_abc
2,A..1,initial..1,PRIM..1,A..nn.1,A,nn,nn_vn_alfa_abc
3,à..1,per..1,PRIM..1,á..pp.1,á,pp,pp_i_i
4,à..1,per..1,PRIM..1,à..pp.1,à,pp,pp_i_i
...,...,...,...,...,...,...,...
137123,övre_medelklass..1,medelklass..1,PRIM..1,övre_medelklass..nnm.1,övre medelklass,nnm,nnm_0u0_frid
137124,Övre_Volta..1,Burkina_Faso..1,PRIM..1,Övre_Volta..pmm.1,Övre Volta,pmm,pmm_n0lp_sri_lanka
137125,övrig..1,annan..1,PRIM..1,övrig..pn.1,övrig,pn,pn_o_all
137126,övärld..1,ögrupp..1,PRIM..1,övärld..nn.1,övärld,nn,nn_2u_mening
